# CKD Detection Using XGBoost with Multi-Modal Data

## Overview
This notebook demonstrates a comprehensive approach to Chronic Kidney Disease (CKD) detection using XGBoost classification on multi-modal data sources:

1. **Clinical Data (EHR)**: Lab values, vital signs, diagnosis codes, medications
2. **Claims Data**: Healthcare utilization patterns, insurance coverage
3. **SDOH Data**: Social determinants of health from Census, CDC PLACES, USDA, ADI
4. **Retail Purchase Patterns**: Dietary and health product purchases by geolocation

## Objectives
- Integrate multi-source patient data
- Train XGBoost classifier for CKD detection
- Analyze feature importance across data sources
- Provide clinical interpretability using SHAP
- Generate actionable insights for early intervention

In [ ]:
# Import libraries
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

%matplotlib inline
%load_ext autoreload
%autoreload 2

## 1. Data Generation

Generate synthetic multi-modal patient data for demonstration

In [ ]:
from data_processing.synthetic_data_generator import SyntheticDataGenerator

# Generate synthetic data
generator = SyntheticDataGenerator(n_patients=1000, random_state=42)
clinical_df, claims_df, sdoh_df, retail_df, labels_df = generator.generate_all_data()

print("Data generated successfully!")
print(f"\nClinical data: {clinical_df.shape}")
print(f"Claims data: {claims_df.shape}")
print(f"SDOH data: {sdoh_df.shape}")
print(f"Retail data: {retail_df.shape}")
print(f"Labels: {labels_df.shape}")

In [ ]:
# Explore CKD distribution
print("CKD Stage Distribution:")
print(labels_df['ckd_stage'].value_counts().sort_index())
print(f"\nCKD Prevalence: {labels_df['has_ckd'].mean():.2%}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# CKD stages
labels_df['ckd_stage'].value_counts().sort_index().plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('CKD Stage Distribution', fontsize=14)
axes[0].set_xlabel('CKD Stage', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)

# Binary classification
labels_df['has_ckd'].value_counts().plot(kind='bar', ax=axes[1], color=['green', 'orange'])
axes[1].set_title('Binary CKD Classification', fontsize=14)
axes[1].set_xlabel('Has CKD', fontsize=12)
axes[1].set_ylabel('Count', fontsize=12)
axes[1].set_xticklabels(['No CKD', 'CKD'], rotation=0)

plt.tight_layout()
plt.show()

## 2. Data Exploration

Explore each data source

In [ ]:
# Clinical data overview
print("Clinical Data Sample:")
display(clinical_df.head())

print("\nClinical Data Summary Statistics:")
display(clinical_df[['serum_creatinine', 'egfr', 'albuminuria_acr', 'hba1c', 
                      'systolic_bp', 'diastolic_bp', 'bmi']].describe())

In [ ]:
# Visualize clinical features by CKD status
merged_clinical = clinical_df.merge(labels_df[['patient_id', 'has_ckd']], on='patient_id')

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

features = ['serum_creatinine', 'egfr', 'albuminuria_acr', 'hba1c', 'systolic_bp', 'bmi']

for i, feature in enumerate(features):
    merged_clinical.boxplot(column=feature, by='has_ckd', ax=axes[i])
    axes[i].set_title(f'{feature} by CKD Status', fontsize=12)
    axes[i].set_xlabel('Has CKD', fontsize=10)
    axes[i].set_ylabel(feature, fontsize=10)

plt.suptitle('Clinical Features by CKD Status', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Claims data exploration
print("Claims Data Sample:")
display(claims_df.head())

# Merge with labels
merged_claims = claims_df.merge(labels_df[['patient_id', 'has_ckd']], on='patient_id')

# Compare utilization by CKD status
utilization_features = ['er_visits_count', 'hospital_admissions_count', 
                       'primary_care_visits_count', 'specialist_visits_count']

comparison = merged_claims.groupby('has_ckd')[utilization_features].mean()
print("\nMean Healthcare Utilization by CKD Status:")
display(comparison)

In [ ]:
# SDOH data exploration
print("SDOH Data Sample:")
display(sdoh_df.head())

# Visualize SDOH distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

sdoh_df['median_household_income'].hist(bins=30, ax=axes[0, 0], color='skyblue', edgecolor='black')
axes[0, 0].set_title('Median Household Income Distribution', fontsize=12)
axes[0, 0].set_xlabel('Income ($)', fontsize=10)

sdoh_df['adi_national_percentile'].hist(bins=20, ax=axes[0, 1], color='coral', edgecolor='black')
axes[0, 1].set_title('Area Deprivation Index Distribution', fontsize=12)
axes[0, 1].set_xlabel('ADI Percentile', fontsize=10)

sdoh_df['diabetes_prevalence'].hist(bins=20, ax=axes[1, 0], color='lightgreen', edgecolor='black')
axes[1, 0].set_title('Diabetes Prevalence Distribution', fontsize=12)
axes[1, 0].set_xlabel('Prevalence (%)', fontsize=10)

sdoh_df['food_desert_indicator'].value_counts().plot(kind='bar', ax=axes[1, 1], color='orange')
axes[1, 1].set_title('Food Desert Distribution', fontsize=12)
axes[1, 1].set_xlabel('Food Desert', fontsize=10)
axes[1, 1].set_xticklabels(['No', 'Yes'], rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# Retail data exploration
print("Retail Purchase Data Sample:")
display(retail_df.head())

# Merge with labels
merged_retail = retail_df.merge(labels_df[['patient_id', 'has_ckd']], on='patient_id')

# Compare dietary patterns by CKD status
dietary_features = ['processed_food_purchases', 'fresh_produce_purchases', 
                   'high_sodium_food_purchases', 'health_conscious_score']

dietary_comparison = merged_retail.groupby('has_ckd')[dietary_features].mean()
print("\nMean Dietary Patterns by CKD Status:")
display(dietary_comparison)

## 3. Data Integration & Preprocessing

In [ ]:
from data_processing.data_integration import DataIntegrationPipeline

# Initialize pipeline
pipeline = DataIntegrationPipeline()

# Integrate and preprocess data
X, y = pipeline.prepare_for_modeling(
    clinical_df=clinical_df,
    claims_df=claims_df,
    sdoh_df=sdoh_df,
    retail_df=retail_df,
    labels_df=labels_df,
    fit=True
)

print(f"\nFinal feature matrix: {X.shape}")
print(f"Target variable: {y.shape}")
print(f"\nFeatures: {pipeline.feature_names[:10]}... ({len(pipeline.feature_names)} total)")

## 4. XGBoost Model Training

In [ ]:
from models.xgboost_model import CKDXGBoostClassifier

# Initialize model
model = CKDXGBoostClassifier(
    model_type='binary',
    use_smote=True,
    random_state=42
)

# Train model
results = model.train(
    X=X,
    y=y,
    test_size=0.2,
    tune_hyperparams=False,  # Set to True for hyperparameter tuning
    cv_folds=5
)

print("\nModel training complete!")

In [ ]:
# Display results
print("\n=== Training Set Performance ===")
for metric, value in results['train'].items():
    if metric not in ['confusion_matrix', 'classification_report']:
        print(f"{metric}: {value:.4f}")

print("\n=== Test Set Performance ===")
for metric, value in results['test'].items():
    if metric not in ['confusion_matrix', 'classification_report']:
        print(f"{metric}: {value:.4f}")

print("\n=== Classification Report (Test Set) ===")
print(results['test']['classification_report'])

## 5. Feature Importance Analysis

In [ ]:
# Get feature importance
feature_importance = model.get_feature_importance(top_n=30)

print("Top 30 Most Important Features:")
display(feature_importance)

In [ ]:
# Plot feature importance
model.plot_feature_importance(top_n=20)

## 6. Model Evaluation & Interpretability (SHAP)

In [ ]:
from sklearn.model_selection import train_test_split
from evaluation.model_evaluation import ModelEvaluator

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Get predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Initialize evaluator
evaluator = ModelEvaluator(model.model, pipeline.feature_names)

In [ ]:
# Confusion Matrix
model.plot_confusion_matrix(y_test, y_pred)

In [ ]:
# ROC Curve
evaluator.plot_roc_curve(y_test, y_proba)

In [ ]:
# Precision-Recall Curve
evaluator.plot_precision_recall_curve(y_test, y_proba)

In [ ]:
# SHAP Summary Plot
evaluator.plot_shap_summary(X_test)

In [ ]:
# SHAP Bar Plot
evaluator.plot_shap_bar(X_test, top_n=20)

In [ ]:
# Feature importance by data source group
group_analysis = evaluator.plot_group_importance(feature_importance)

In [ ]:
# Display group analysis
print("\nFeature Importance by Data Source:")
for group_name, data in sorted(group_analysis.items(), 
                               key=lambda x: x[1]['total_importance'], 
                               reverse=True):
    print(f"\n{group_name}:")
    print(f"  Total Importance: {data['total_importance']:.4f}")
    print(f"  Mean Importance:  {data['mean_importance']:.4f}")
    print(f"  Feature Count:    {data['feature_count']}")
    print(f"  Top Features:     {', '.join(data['top_features'])}")

## 7. Clinical Insights

In [ ]:
# Generate clinical insights
insights = evaluator.generate_clinical_insights(feature_importance, top_n=15)

print("\n=== TOP PREDICTIVE FEATURES WITH CLINICAL INTERPRETATION ===")
for i, feature in enumerate(insights['top_predictors'], 1):
    importance = insights['feature_importance_scores'][feature]
    interpretation = insights['clinical_interpretation'].get(feature, 'N/A')
    print(f"\n{i}. {feature} (importance: {importance:.4f})")
    if interpretation != 'N/A':
        print(f"   → {interpretation}")

## 8. Key Findings & Conclusions

### Multi-Modal Data Integration
- Successfully integrated **4 distinct data sources**: Clinical (EHR), Claims, SDOH, and Retail
- Created **comprehensive patient profiles** combining medical and behavioral data

### Model Performance
- XGBoost achieved strong predictive performance for CKD detection
- Clinical features remain the primary predictors (eGFR, creatinine, albuminuria)
- SDOH and retail data provide **additional risk stratification**

### Clinical Insights
1. **Traditional biomarkers** (eGFR, creatinine) are most important
2. **Healthcare utilization patterns** signal disease progression
3. **Socioeconomic factors** (ADI, poverty) impact CKD risk
4. **Dietary patterns** (from retail data) correlate with CKD outcomes

### Future Directions
- Integration of **temporal patterns** (longitudinal data)
- **Real-world validation** with de-identified patient data
- **Early intervention** targeting high-risk populations identified by the model
- **Policy implications** based on SDOH findings

## 9. Save Model for Production

In [ ]:
# Save trained model
model.save_model('../models/ckd_xgboost_model.pkl')

# Save data pipeline
import joblib
joblib.dump(pipeline, '../models/data_pipeline.pkl')

print("Model and pipeline saved successfully!")
print("  - Model: models/ckd_xgboost_model.pkl")
print("  - Pipeline: models/data_pipeline.pkl")